In [1]:
%pip install langchain openai

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/23/98/c70fac0f1b3193ced86013b563119c27c68ac26b684815f407555224108d/langchain-0.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/6a/54/e0af4b74ebb732bfa9bc83d3e49e577d4e332990742a9ecbe228c532a02d/openai-1.7.2-py3-none-any.whl.metadata
  Obtaining dependency information for jsonpatch<2.0,>=1.33 from https://files.pythonhosted.org/packages/73/07/02e16ed01e04a374e644b575638ec7987ae846d25ad97bcc9945a3ee4b0e/jsonpatch-1.33-py2.py3-none-any.whl.metadata
  Obtaining dependency information for langchain-community<0.1,>=0.0.9 from https://files.pythonhosted.org/packages/2a/24/224115d2666ff3ff4917f2ebc98b9da2d2fe8835ec57909ceca5972a117e/langchain_community-0.0.13-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<0.2,>=0.1.7 from https://files.pythonhosted.org/packages/5c/b9/95f91c284a9c316dbc1c6147076dee

# Elements of a prompt

In [2]:
import os

os.environ['OPENAI_API_KEY'] = ''

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

chat_model = ChatOpenAI()

conversation_chain = ConversationChain(
    llm=chat_model
)

print(conversation_chain.prompt.template)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


# Few-shot learning

In [4]:
from langchain.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate,
)

examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "5+6", "output": "11"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())


Human: 2+2
AI: 4
Human: 2+3
AI: 5
Human: 5+6
AI: 11


In [5]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are wonderous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

print(final_prompt.format(input='What is 3+3?'))

System: You are wonderous wizard of math.
Human: 2+2
AI: 4
Human: 2+3
AI: 5
Human: 5+6
AI: 11
Human: What is 3+3?


In [6]:
from langchain.chains import LLMChain

chain = LLMChain(
    llm=chat_model,
    prompt=final_prompt,
    verbose=True
)

chain.run("What is 3+3?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are wonderous wizard of math.
Human: 2+2
AI: 4
Human: 2+3
AI: 5
Human: 5+6
AI: 11
Human: What is 3+3?


/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



> Finished chain.


'3+3 is equal to 6.'

# Memetic Proxy

In [7]:
from langchain.prompts import PromptTemplate

template = """
System: {reference}
Provide an helpful answer to the following question

Human: {question}

AI:"""

prompt = PromptTemplate.from_template(template)

chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    verbose=True
)

high_level = "Imagine you are a Professor teaching at the PhD level."
lower_level = "Imagine you are a Kindergarten teacher."


question = "What is Quantum Mechanics"

chain.run({
    'question': question,
    'reference': high_level  
})



> Entering new LLMChain chain...
Prompt after formatting:

System: Imagine you are a Professor teaching at the PhD level.
Provide an helpful answer to the following question

Human: What is Quantum Mechanics

AI:

> Finished chain.


'Quantum mechanics is a branch of physics that deals with the behavior of particles at the atomic and subatomic levels. It is a fundamental theory that describes the nature of matter and energy, explaining phenomena such as the wave-particle duality, superposition, and entanglement. Quantum mechanics provides a mathematical framework to understand and predict the behavior of microscopic particles, challenging our classical intuition. It is widely used in various fields, including particle physics, condensed matter physics, and quantum information science. By studying quantum mechanics, we gain a deeper understanding of the fundamental building blocks of the universe and can develop technologies based on quantum principles, such as quantum computing and quantum cryptography.'

In [8]:
chain.run({
    'question': question,
    'reference': lower_level  
})



> Entering new LLMChain chain...
Prompt after formatting:

System: Imagine you are a Kindergarten teacher.
Provide an helpful answer to the following question

Human: What is Quantum Mechanics

AI:

> Finished chain.


'Quantum Mechanics is a branch of physics that deals with the behavior of particles at the atomic and subatomic level. It explains how tiny particles, such as electrons and photons, can exist in multiple states or locations at the same time, known as superposition. Additionally, Quantum Mechanics also describes the phenomenon of particles being entangled, where the state of one particle is instantaneously connected to another particle, regardless of the distance between them. This field of study has led to many groundbreaking technological advancements, such as quantum computing and cryptography. However, the concepts of Quantum Mechanics can be quite complex and require a deep understanding of mathematics and physics to fully comprehend.'

# Chain of Thoughts

In [9]:
import pandas as pd

file_path = '/kaggle/input/chain-of-thought-collection/CoT_collection.json'

cot_df = pd.read_json(file_path)
 
cot_df

,407496,897822,1639180,1268206,112965,176714,1365585,1266292,306879,765453,...,1155268,891128,558286,1432572,471785,69445,1810512,1472056,570474,359628
source,Article: Phytochemistry is a branch of plant b...,"In this task, you are provided with an article...",What's the answer to that question: where did ...,Extract the answer to the question from the fo...,"Which entity is this text about?\n\nRichard ""R...",The town lies along the stretch of coastline d...,Two sisters went to the hospital. The medics r...,"Question: ""What else does the pigment make red...","""Mama came over and sat down beside Alice. Gen...","Given a travel related situation, and given a ...",...,Read the given text and if it has abusive cont...,"In this task, you will be shown a prompt from ...","In this task, you're given the title of a five...",Read the following context and answer the ques...,You will be given a passage consisting of set ...,Answer the following question:\n\nthe person w...,Exercise: read the text and answer the questio...,Just minutes after Donald Trump fired Rex Till...,"In this task, you're given four sentences of a...",Read this article and answer this question The...
target,from plants,Regulation,Battle of Appomattox Court House,major heart attacks,Red Skelton,no,Sue,lips,no,Where to watch German world cup soccer games i...,...,no,holding this court may affirm on any grounds s...,a,The narrator feels insecure about her own faith .,kitchen,designer,True,A,A,4
rationale,The article states that many medicinal and rec...,This act has all the features of Regulation. T...,The Battle of Appomattox Court House was a bat...,The answer is the number of major heart attack...,The text is about Red Skelton. The context des...,The paragraph doesn't mention the date of the ...,Sickle cell trait is a hereditary disorder in ...,The context mentions that the pigment pheomela...,The premise states that Mama came over and sat...,The question asks for a location to watch Germ...,...,"The given text does not contain any abuse, neg...","The first option is ""holding this court may af...",Sentence a is not connected with the rest of t...,The narrator mentions that she grew up in a no...,We can infer from the last fact in the passage...,"The answer is ""designer""\nA database designer ...",The text explains that bumpers are legally req...,The question of what Tillerson said Trump shou...,"Sentence4 shows that the man is ill and sad, s...","To solve the problem, we need to find a month ..."
config,none,task_744,none,dbert,none,none,none,dbert,none,task_1639,...,task_108,task_287,task_214,none,task_083,none,boolq,none,task_296,none
task,squad_v1,eurlex,web_questions,adversarial_qa@dbert,quac,rte,ropes,adversarial_qa@dbert,wnli,diqa,...,cad,casehold,roc_stories,cosmos_qa,babi,natural_questions,super_glue@boolq,quail,roc_stories,drop
prompt,6,none,whats_the_answer,based_on,8,1,plain_no_background,question_context_answer,4,none,...,none,none,none,description_context_question_text,none,2,exercise,context_question_description_answer_id,none,4


In [10]:
cot_df = cot_df.T
cot_df = cot_df[cot_df.task == 'ai2_arithmetic_questions']
cot_df

,source,target,rationale,config,task,prompt
815888,You are provided with an arithmetic question. ...,0.6,Since Blake filled the bucket with 0.8 gallon ...,task_745,ai2_arithmetic_questions,none
815685,You are provided with an arithmetic question. ...,0.5,0.2 + 0.3 = 0.5,task_745,ai2_arithmetic_questions,none
816139,You are provided with an arithmetic question. ...,0.9,Craig walked 0.2 mile from school to David's h...,task_745,ai2_arithmetic_questions,none
815822,You are provided with an arithmetic question. ...,0.9,Hannah's Vegetarian Restaurant bought 0.3 poun...,task_745,ai2_arithmetic_questions,none
815930,You are provided with an arithmetic question. ...,12.3,Alyssa spent $5.71 on football and $6.59 on ma...,task_745,ai2_arithmetic_questions,none
...,...,...,...,...,...,...
816034,You are provided with an arithmetic question. ...,0.3,"Ella owns 2 dogs. Each day, 1 dog eats 0.175 s...",task_745,ai2_arithmetic_questions,none
815781,You are provided with an arithmetic question. ...,2.0,The problem is: 6 + 4 - 4 / 2.0 = 10.0 - 2.0 =...,task_745,ai2_arithmetic_questions,none
815946,You are provided with an arithmetic question. ...,34.7,Mary loves eating fruits. Mary paid $11.08 for...,task_745,ai2_arithmetic_questions,none
815701,You are provided with an arithmetic question. ...,81.0,Joan has 40 blue balloons Melanie has 41 blue ...,task_745,ai2_arithmetic_questions,none


In [11]:
cot_df.reset_index(inplace=True, drop=True)
cot_df

,source,target,rationale,config,task,prompt
0,You are provided with an arithmetic question. ...,0.6,Since Blake filled the bucket with 0.8 gallon ...,task_745,ai2_arithmetic_questions,none
1,You are provided with an arithmetic question. ...,0.5,0.2 + 0.3 = 0.5,task_745,ai2_arithmetic_questions,none
2,You are provided with an arithmetic question. ...,0.9,Craig walked 0.2 mile from school to David's h...,task_745,ai2_arithmetic_questions,none
3,You are provided with an arithmetic question. ...,0.9,Hannah's Vegetarian Restaurant bought 0.3 poun...,task_745,ai2_arithmetic_questions,none
4,You are provided with an arithmetic question. ...,12.3,Alyssa spent $5.71 on football and $6.59 on ma...,task_745,ai2_arithmetic_questions,none
...,...,...,...,...,...,...
575,You are provided with an arithmetic question. ...,0.3,"Ella owns 2 dogs. Each day, 1 dog eats 0.175 s...",task_745,ai2_arithmetic_questions,none
576,You are provided with an arithmetic question. ...,2.0,The problem is: 6 + 4 - 4 / 2.0 = 10.0 - 2.0 =...,task_745,ai2_arithmetic_questions,none
577,You are provided with an arithmetic question. ...,34.7,Mary loves eating fruits. Mary paid $11.08 for...,task_745,ai2_arithmetic_questions,none
578,You are provided with an arithmetic question. ...,81.0,Joan has 40 blue balloons Melanie has 41 blue ...,task_745,ai2_arithmetic_questions,none


In [12]:
cot_df = cot_df[['source', 'target', 'rationale']]
cot_df

,source,target,rationale
0,You are provided with an arithmetic question. ...,0.6,Since Blake filled the bucket with 0.8 gallon ...
1,You are provided with an arithmetic question. ...,0.5,0.2 + 0.3 = 0.5
2,You are provided with an arithmetic question. ...,0.9,Craig walked 0.2 mile from school to David's h...
3,You are provided with an arithmetic question. ...,0.9,Hannah's Vegetarian Restaurant bought 0.3 poun...
4,You are provided with an arithmetic question. ...,12.3,Alyssa spent $5.71 on football and $6.59 on ma...
...,...,...,...
575,You are provided with an arithmetic question. ...,0.3,"Ella owns 2 dogs. Each day, 1 dog eats 0.175 s..."
576,You are provided with an arithmetic question. ...,2.0,The problem is: 6 + 4 - 4 / 2.0 = 10.0 - 2.0 =...
577,You are provided with an arithmetic question. ...,34.7,Mary loves eating fruits. Mary paid $11.08 for...
578,You are provided with an arithmetic question. ...,81.0,Joan has 40 blue balloons Melanie has 41 blue ...


In [13]:
cot_df.rename(
    columns = {
        'source': 'Question',
        'target': 'Response',
        'rationale': 'Rationale'
    }, 
    inplace=True)
cot_df

/tmp/ipykernel_42/1618394725.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cot_df.rename(


,Question,Response,Rationale
0,You are provided with an arithmetic question. ...,0.6,Since Blake filled the bucket with 0.8 gallon ...
1,You are provided with an arithmetic question. ...,0.5,0.2 + 0.3 = 0.5
2,You are provided with an arithmetic question. ...,0.9,Craig walked 0.2 mile from school to David's h...
3,You are provided with an arithmetic question. ...,0.9,Hannah's Vegetarian Restaurant bought 0.3 poun...
4,You are provided with an arithmetic question. ...,12.3,Alyssa spent $5.71 on football and $6.59 on ma...
...,...,...,...
575,You are provided with an arithmetic question. ...,0.3,"Ella owns 2 dogs. Each day, 1 dog eats 0.175 s..."
576,You are provided with an arithmetic question. ...,2.0,The problem is: 6 + 4 - 4 / 2.0 = 10.0 - 2.0 =...
577,You are provided with an arithmetic question. ...,34.7,Mary loves eating fruits. Mary paid $11.08 for...
578,You are provided with an arithmetic question. ...,81.0,Joan has 40 blue balloons Melanie has 41 blue ...


In [14]:
list(cot_df.T.to_dict().values())[0]

{'Question': "You are provided with an arithmetic question. Your task is to compute the solution using the given arithmetic operations. The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.\n\nBlake filled a bucket with 0.8 gallon of water. Later, he poured out 0.2 gallon of the water. How much water is in the bucket?",
 'Response': '0.6',
 'Rationale': 'Since Blake filled the bucket with 0.8 gallon of water, and poured out 0.2 gallon of it, the current amount of water left in the bucket is 0.8 - 0.2 = 0.6 gallon'}

In [15]:
import random

random.sample(list(cot_df.T.to_dict().values()), 10)

[{'Question': "You are provided with an arithmetic question. Your task is to compute the solution using the given arithmetic operations. The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.\n\nHannah's Vegetarian Restaurant bought 0.3 poundof green peppers and 0.6 pound of red peppers.How many pounds of peppers did Hannah's Vegetarian Restaurantbuy in all?",
  'Response': '0.9',
  'Rationale': 'Hannah\'s Vegetarian Restaurant bought 0.3 poundof green peppers and 0.6 pound of red peppers.Therefore, it bought (0.3) + (0.6) = 0.9 pounds in all."'},
 {'Question': "You are provided with an arithmetic question. Your task is to compute the solution using the given arithmetic operations. The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.\n\nJenny ran 0.6 mile and walked 0.4 mile. How much farther did 

In [16]:
from langchain.prompts import (
    FewShotPromptTemplate,
    PromptTemplate
)

examples = random.sample(list(cot_df.T.to_dict().values()), 10)


template = """
Question: {Question}
Rationale: {Rationale}
Response: {Response}
"""

example_prompt = PromptTemplate(
    input_variables=["Question", "Rationale", "Response"], 
    template=template
)

system_prompt = """
You are provided with an arithmetic question. 
Your task is to compute the solution using the given arithmetic operations. 
The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). 
The answer should be correct to one decimal place.
"""

COT_prompt = FewShotPromptTemplate(
    prefix=system_prompt,
    example_prompt=example_prompt,
    examples=examples,
    suffix="Question: {input}", 
    input_variables=['input']
)


print(COT_prompt.format(input='[EXAMPLE QUESTION]'))


You are provided with an arithmetic question. 
Your task is to compute the solution using the given arithmetic operations. 
The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). 
The answer should be correct to one decimal place.



Question: You are provided with an arithmetic question. Your task is to compute the solution using the given arithmetic operations. The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.

At Lindsey's Vacation Wear, 0.37 the garments are bikinis and 0.23 are trunks. What fraction of the garments are either bikinis or trunks?
Rationale: The fraction of garments that are bikinis is 0.37 and the fraction of garments that are trunks is 0.23, so the total fraction of garments that are either bikinis or trunks is 0.37 + 0.23 = 0.60.
Response: 0.6



Question: You are provided with an arithmetic question. Your task is to comp

In [17]:
chain = LLMChain(
    llm=chat_model,
    prompt=COT_prompt,
    verbose=True
)

question = """
Mandy made an apple pie. She used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg. 
How much more cinnamon than nutmeg did Mandy use?
"""

chain.run(question)



> Entering new LLMChain chain...
Prompt after formatting:

You are provided with an arithmetic question. 
Your task is to compute the solution using the given arithmetic operations. 
The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). 
The answer should be correct to one decimal place.



Question: You are provided with an arithmetic question. Your task is to compute the solution using the given arithmetic operations. The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.

At Lindsey's Vacation Wear, 0.37 the garments are bikinis and 0.23 are trunks. What fraction of the garments are either bikinis or trunks?
Rationale: The fraction of garments that are bikinis is 0.37 and the fraction of garments that are trunks is 0.23, so the total fraction of garments that are either bikinis or trunks is 0.37 + 0.23 = 0.60.
Response: 0.6



Question: You ar

'Rationale: 0.6 - 0.5 = 0.1\nResponse: 0.1'

# Self-consistency

In [18]:
chat_model = ChatOpenAI(temperature=0.7)

cot_chain = LLMChain(
    llm=chat_model,
    prompt=COT_prompt,
    verbose=True
)


template = """
Using the following answers, return the answer that comes up more often

{answers}

ANSWER:"""

consistency_prompt = PromptTemplate.from_template(template)

consistency_chain = LLMChain(
    llm=chat_model,
    prompt=consistency_prompt,
    verbose=True
)

question = """
Mandy made an apple pie. She used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg. 
How much more cinnamon than nutmeg did Mandy use?
"""

responses = []

for i in range(10):
    response = cot_chain.run(question)
    responses.append(response)



> Entering new LLMChain chain...
Prompt after formatting:

You are provided with an arithmetic question. 
Your task is to compute the solution using the given arithmetic operations. 
The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). 
The answer should be correct to one decimal place.



Question: You are provided with an arithmetic question. Your task is to compute the solution using the given arithmetic operations. The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.

At Lindsey's Vacation Wear, 0.37 the garments are bikinis and 0.23 are trunks. What fraction of the garments are either bikinis or trunks?
Rationale: The fraction of garments that are bikinis is 0.37 and the fraction of garments that are trunks is 0.23, so the total fraction of garments that are either bikinis or trunks is 0.37 + 0.23 = 0.60.
Response: 0.6



Question: You ar

In [19]:
answers = '\n\n'.join(responses) 
consistency_chain.run(answers)



> Entering new LLMChain chain...
Prompt after formatting:

Using the following answers, return the answer that comes up more often

Rationale: 0.6 - 0.5 = 0.1
Response: 0.1

Rationale: 0.6 - 0.5 = 0.1
Response: 0.1

Rationale: 0.6 - 0.5 = 0.1 
Response: 0.1

Rationale: 0.6 - 0.5 = 0.1
Response: 0.1

Rationale: Mandy used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg. The difference in the amount used is 0.6 - 0.5 = 0.1 tablespoon.
Response: 0.1

Rationale: 0.6 - 0.5 = 0.1
Response: 0.1

Rationale: 0.6 - 0.5 = 0.1
Response: 0.1

Rationale: 0.6 - 0.5 = 0.1
Response: 0.1

Rationale: 0.6 - 0.5 = 0.1
Response: 0.1

Rationale: 0.6 - 0.5 = 0.1
Response: 0.1

ANSWER:

> Finished chain.


'0.1'

# Inception

In [20]:
system_prompt = """
You are provided with an arithmetic question. 
Your task is to compute the solution using the given arithmetic operations. 
The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). 
The answer should be correct to one decimal place.
"""

template = """

{input}

Let's think about it step by step:
"""

prompt = PromptTemplate.from_template(system_prompt + template)

chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    verbose=True
)

question = """
Mandy made an apple pie. She used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg. 
How much more cinnamon than nutmeg did Mandy use?
"""

chain.run(question)



> Entering new LLMChain chain...
Prompt after formatting:

You are provided with an arithmetic question. 
Your task is to compute the solution using the given arithmetic operations. 
The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). 
The answer should be correct to one decimal place.



Mandy made an apple pie. She used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg. 
How much more cinnamon than nutmeg did Mandy use?


Let's think about it step by step:


> Finished chain.


'1. Mandy used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg.\n2. To find out how much more cinnamon than nutmeg Mandy used, we need to subtract the amount of nutmeg from the amount of cinnamon.\n3. So, the calculation would be: 0.6 - 0.5 = 0.1\n4. Therefore, Mandy used 0.1 tablespoon more cinnamon than nutmeg.'

# Self-Ask

In [21]:
%pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32003 sha256=f6cf25460302da44598e80f2b9d564154b53ca4cdd4a0b987273f79bf873fb5e
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results
Note: you may need to restart the kernel to use updated packages.


In [22]:
os.environ['SERPAPI_API_KEY'] = ''

In [23]:
from langchain.agents import (
    initialize_agent,
    AgentType,
    load_tools
)

tools = load_tools(["serpapi"])

tools[0].name = 'Intermediate Answer'

self_ask_with_search = initialize_agent(
    tools=tools, 
    llm=chat_model, 
    agent=AgentType.SELF_ASK_WITH_SEARCH, 
    verbose=True,
    handle_parsing_errors=True
)

print(self_ask_with_search.agent.llm_chain.prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [24]:
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()

question = "What is the hometown of the reigning men's U.S. Open champion?"

self_ask_with_search.run(
    question,
    callbacks=[handler]
)



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mot

'Invalid or incomplete response'

# ReAct

In [25]:

tools = load_tools(["serpapi", "llm-math"], llm=chat_model)
agent = initialize_agent(
    tools=tools, 
    llm=chat_model, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

handler = StdOutCallbackHandler()

question = """
Who is Leo DiCaprio's girlfriend? 
What is her current age raised to the 0.43 power?
"""

agent.run(
    question,
    callbacks=[handler]
)



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 
Who is Leo DiCaprio's girlfriend? 
What is her current age raised to the 0.43 power?

Thought:

> Finished chain.
I need to find out who Leo DiCaprio's girlfriend i

"Vittoria Ceretti's current age raised to the 0.43 power is approximately 3.9913."

In [26]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


# Plan and execute

In [27]:
%pip install langchain-experimental

  Obtaining dependency information for langchain-experimental from https://files.pythonhosted.org/packages/01/61/9d8a9ed481bf245ca5b678c211488e1b51c7c4e5441a5923ee8a4eee248c/langchain_experimental-0.0.49-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.7/165.7 kB 5.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [28]:
from langchain_experimental.plan_and_execute import (
    PlanAndExecute, 
    load_agent_executor, 
    load_chat_planner
)

chat_model = ChatOpenAI(model_name='gpt-4')

tools = load_tools(["serpapi", "llm-math"], llm=chat_model)

planner = load_chat_planner(chat_model)

executor = load_agent_executor(
    chat_model, 
    tools, 
    verbose=True
)
agent = PlanAndExecute(
    planner=planner, 
    executor=executor, 
    verbose=True
)

In [29]:
print(planner.llm_chain.prompt.format(input='Some input'))

System: Let's first understand the problem and devise a plan to solve the problem. Please output the plan starting with the header 'Plan:' and then followed by a numbered list of steps. Please make the plan the minimum number of steps required to accurately complete the task. If the task is a question, the final step should almost always be 'Given the above steps taken, please respond to the users original question'. At the end of your plan, say '<END_OF_PLAN>'
Human: Some input


In [30]:
print(executor.chain.agent.llm_chain.prompt.format(
    previous_steps='previous_steps', 
    current_step='current_step', 
    agent_scratchpad='agent_scratchpad'
))

System: Respond to the human as helpfully and accurately as possible. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query., args: {'tool_input': {'type': 'string'}}
Calculator: Useful for when you need to answer questions about math., args: {'tool_input': {'type': 'string'}}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or Search, Calculator

Provide only ONE action per $JSON_BLOB, as shown:

```
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "Final response to human"
}

In [31]:
question ="""
Who is Leo DiCaprio's girlfriend? 
What is her current age raised to the 0.43 power?
"""
agent.run(question)



> Entering new PlanAndExecute chain...
steps=[Step(value='Research the current girlfriend of Leonardo DiCaprio.'), Step(value='Find out her current age.'), Step(value='Calculate her age raised to the power of 0.43.'), Step(value="Given the above steps taken, please respond to the user's original question.\n\n")]

> Entering new AgentExecutor chain...
Thought: I need to use the search engine tool to find the current girlfriend of Leonardo DiCaprio.
Action:
```
{
  "action": "Search",
  "action_input": "current girlfriend of Leonardo DiCaprio"
}
```
Observation: Vittoria Ceretti
Thought:The latest search results show that the current girlfriend of Leonardo DiCaprio is Vittoria Ceretti. 
Action:
```
{
  "action": "Final Answer",
  "action_input": "The current girlfriend of Leonardo DiCaprio is Vittoria Ceretti."
}
```

> Finished chain.
*****

Step: Research the current girlfriend of Leonardo DiCaprio.

Response: The current girlfriend of Leonardo DiCaprio is Vittoria Ceretti.

> Enteri

"The age of Vittoria Ceretti, Leonardo DiCaprio's current girlfriend, raised to the power of 0.43 is approximately 3.99."